<img src="tomato_apple1.png" style="text-align: center;"><br>
<h1 style="font-family: Arial; font-size:0.6em;color:purple; text-align: center;font-style:bold">Denis Pastory</h1>
<h1 style="font-family: Arial; font-size:0.6em;color:purple; text-align: center;font-style:bold">Graduate Research Assistant</h1>
<h1 style="font-family: Arial; font-size:0.6em;color:purple; text-align: center;font-style:bold">Laboratory of Information Network</h1>
<h1 style="font-family: Arial; font-size:0.6em;color:purple; text-align: center;font-style:bold">Tokyo University of Agriculture</h1>


<h1 style="font-family: Arial; font-size:2.6em;color:purple; font-style:bold; text-align: center;"><br>
A Real-time Environmental Monitoring and Analysis<br>
For a Simplified Smart Agriculture System</h1>


<h1 style="font-family: Arial; font-size:2.65em;color:purple; font-style:bold">Objective</h1>
<p style="font-family: Arial; font-size:1.50em;color:#4462C0; font-style:bold">  
    - to show how real-time wireless sensor data is used for crop environment monitoring<br><br>
    
    - To analyze crop development stages in greenhouse<br><br></p>

<p style="font-family: Arial;text-align: center; font-size:1.70em;color:#2462C0; font-style:bold">Wireless Sensor in Tomato Greenhouse </p>
<img src="sensor.png" style="width: 800px; height: 500px; padding-left: 6cm;">

In [1]:
import PIL
from PIL import Image
basewidth = 850
img = Image.open('netatmo.png')
wpercent = (basewidth / float(img.size[0]))
hsize = int((float(img.size[1]) * float(wpercent)))
img = img.resize((basewidth, hsize), PIL.Image.ANTIALIAS)
img.save('resized_netatmo.png')

<p style="font-family: Arial;text-align: center; font-size:1.30em;color:#2462C0; font-style:bold">Wireless Sensor Dashboard </p>
<img src="resized_netatmo.png" style="width: 950px; height: 500px; padding-left: 7cm;">

<p style="font-family: Arial;text-align: center; font-size:1.70em;color:#2462C0; font-style:bold">Daily Temp Table </p>
<img src="daily_sensor.png" style="width: 950px; height: 500px; padding-left: 5cm;">

<p style="font-family: Arial;text-align: center; font-size:1.70em;color:#2462C0; font-style:bold">Web Database - Daily Activity input</p>
<img src="web_db.png" style="width: 900px; height: 500px; padding-left: 5cm;">

<p style="font-family: Arial;text-align: center; font-size:1.70em;color:#2462C0; font-style:bold">Distribution of Sensors</p>
<img src="toguPlan.png" style="width: 950px; height: 600px; padding-left: 5cm;">

<p style="font-family: Arial;text-align: center; font-size:1.70em;color:#2462C0; font-style:bold">Crop calender Table </p>
<img src="crop_calender1.png" style="width: 950px; height: 500px; padding-left: 5cm;">


<h1 style="font-family: Arial; font-size:2.65em;color:purple; font-style:bold">STEPS</h1>
<p style="font-family: Arial; font-size:1.40em;color:#4462C0; font-style:bold"> 
- Load all modules<br>
- Get data from  Database<br>
- Query specific sensors, Block, specie(varierty)<br>
- get df  between specific growth period<br>
- Computation of Growing Degree Days (GDD) / cumulative GDD<br>
- Plot our results<br>
</p>

## Load all modules 

In [2]:
import sys 
import pymysql
import  datetime
import numpy as np
import pandas as  pd
import  datetime as   dt
from  calendar import timegm
from bokeh.io import export_png
from IPython.display import IFrame
from   time import  gmtime, strftime
from  bokeh.io   import   output_notebook
from bokeh.models.widgets import Tabs, Panel
from bokeh.layouts import row, column, gridplot
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource,HoverTool,Range1d

In [3]:
conn = pymysql.connect(host = 'xxxxx', port= xxxx, user ='xxxx', passwd = 'xxxxx', db = 'xxxx')


### connect to database


In [4]:
cursor1 = conn.cursor()
cursor2 = conn.cursor()

cursor1.execute('SELECT sdate,ROUND((max_temp+min_temp)/2,1), sensor FROM daily_temp ORDER BY sdate ASC')
cursor2.execute('SELECT DISTINCT block,vid,planting, harvesting FROM toguchi_plant WHERE  planting IN (SELECT Planting FROM toguchi_plant) ORDER BY planting DESC')

result1 = cursor1.fetchall()
result2 = cursor2.fetchall()

df1 = pd.DataFrame( [[ij for ij in i] for i in result1] )
df2 = pd.DataFrame( [[ij for ij in i] for i in result2] )

df1.rename(columns={0: 'Date', 1:'Mean_T', 2:'sensor'}, inplace=True);
df2.rename(columns={0: 'block', 1: 'vid', 2: 'startPlant', 3: 'startHarv'}, inplace=True);
if cursor1 and conn:
    cursor1.close()
if cursor2 and conn:
    cursor2.close()
    conn.close()    

### let's view our data

In [5]:
df2.head()

,block,vid,startPlant,startHarv
0,C,v1,2017-07-18,2017-09-18
1,D,v1,2017-05-26,2017-07-25
2,A,v2,2017-04-27,2017-06-19
3,E,v1,2017-03-17,None
4,F,v1,2017-02-21,2017-05-08


In [6]:
df1.head()

,Date,Mean_T,sensor
0,2017-04-28,23.2,02:00:00:28:8c:e6
1,2017-04-28,23.2,03:00:00:02:c5:f8
2,2017-04-28,23.2,03:00:00:05:05:30
3,2017-04-29,21.9,02:00:00:28:8c:e6
4,2017-04-29,21.9,03:00:00:02:c5:f8


### Query Specific Sensor in Block

In [7]:
df2_A = df2.query('block == "A" & vid == "v2"')
clean_df2_A = df2_A.drop(['block','vid'], 1)
df1_A = df1.query('sensor == "02:00:00:28:8c:e6"')
clean_df1_A = df1_A.drop('sensor',1)

df2_C = df2.query('block == "C" & vid == "v1"')
clean_df2_C = df2_C.drop(['block','vid'], 1)
df1_C= df1.query('sensor == "03:00:00:05:05:30"')
clean_df1_C = df1_C.drop('sensor',1)

df2_D = df2.query('block == "D" & vid == "v1"')
clean_df2_D = df2_D.drop(['block','vid'], 1)
df1_D = df1.query('sensor == "03:00:00:02:c5:f8"')
clean_df1_D = df1_D.drop('sensor',1)

In [8]:
clean_df2_C.head()

,startPlant,startHarv
0,2017-07-18,2017-09-18
7,2012-08-16,None
18,None,2010-03-16


In [9]:
clean_df1_A.head()

,Date,Mean_T
0,2017-04-28,23.2
3,2017-04-29,21.9
6,2017-04-30,23.9
9,2017-05-01,24.8
12,2017-05-02,23.1


### ...now get df between certain growth period

In [10]:
def df1_df2(df11,df22):
    """
    Take in two dataframes, from df22 we get the start and end date for date range. 
    The date range will get a complete df from df11
    """
    startHarv_A= df22['startPlant'].iloc[0]
    endHarv = dt.date.today()  
    
    full_range = pd.date_range(startHarv_A, endHarv)

    df11.set_index(['Date'], inplace=True)
    df_complete = df11.sort_index().reindex(full_range).interpolate(method='linear')
    return df_complete

In [11]:
df_intepl = df1_df2(clean_df1_A,clean_df2_A)
df_intep3 = df1_df2(clean_df1_C,clean_df2_C)
df_intep4 = df1_df2(clean_df1_D,clean_df2_D)
df_intep3.head()

,Mean_T
2017-07-18,28.5
2017-07-19,29.8
2017-07-20,28.9
2017-07-21,30.5
2017-07-22,30.6


<p style="font-family: Arial; font-size:2.5em;color:purple; text-align: center;font-style:bold">Growing Degree Days</p>
<h2 style="font-family: Arial; font-size:1.7em;color:#2462C0; font-style:bold;text-align: center;">  
GDD are typically used to monitor crop development,the assumption being 
that development is limited once the temperatures exceed a certain threshold<br></h2>


<p style="font-family: Arial; font-size:2.9em;color:purple; text-align: center;font-style:bold">Growing Degree Days</p>
<img src="gdd1.png" style="width: 800px; height: 300px; padding-left: 5cm;">

### Computation of GDD

In [12]:
# Function to Calculte GDD (Growing Degree Degree)
def compute_gdd (Tmean,Tbase=10.0):
    """
    Growing Degree Days (GDD) calculation based on reference 
    Temp of 10 Deg.Celsius Tmean = (Tmax + Tmin)/2
    
    """
    if Tbase <= Tmean:
        return ((Tmean-Tbase))
    else:
        return (0)

In [13]:
df_intepl['GDD'] = df_intepl.apply(lambda row: compute_gdd(row['Mean_T']),axis=1)
df_intep3['GDD'] = df_intep3.apply(lambda row: compute_gdd(row['Mean_T']),axis=1)
df_intep4['GDD'] = df_intep4.apply(lambda row: compute_gdd(row['Mean_T']),axis=1)

### Computation of cumulative GDD

In [14]:
#Function to calculate Cumulative GDD
def full_df(df22):
    """
    Takes in a df, reset its index and them calculates 
    the cumulative GDD
    
    """
    df22 = df22.reset_index(drop=False)
    df22['Cum_GDD'] = df22.GDD.cumsum()
    df = df22
    return df

In [15]:
dfA = full_df(df_intepl)
dfC = full_df(df_intep3)
dfD = full_df(df_intep4)

In [16]:
dfC.head()

,index,Mean_T,GDD,Cum_GDD
0,2017-07-18,28.5,18.5,18.5
1,2017-07-19,29.8,19.8,38.3
2,2017-07-20,28.9,18.9,57.2
3,2017-07-21,30.5,20.5,77.7
4,2017-07-22,30.6,20.6,98.3


#### Plotting results - Bokeh Library

In [20]:
def plotTab(df,k):
    Date = df.index
    GDD = df.GDD
    Cum_GDD = df.Cum_GDD
    source = ColumnDataSource(data=dict(GDD=GDD,Cum_GDD=Cum_GDD,
                                        Date = pd.to_datetime(df['index'], format='%Y-%m-%d'),)
                             )
    source.add(df['index'].apply(lambda d: d.strftime('%Y-%m-%d')), 'date_formatted') # see line 18
    hover = HoverTool(
        tooltips="""
             <div style="background: #FFFFFF;">
                <span style="font-size: 20px; color: blue;"> cumulative GDD, @Cum_GDD{int}</span><br />
                <span style="font-size: 15px; color: green;"> DATE, @date_formatted</span><br />
            </div>
        """)  
    p = figure(x_axis_label='DATE', x_axis_type='datetime',y_axis_label='CUMULATIVE GDD',
                background_fill_color='#D1D1D8',tools=[hover, 'xpan','reset','previewsave'],plot_height=470,
                logo=None)
    p.line(x='Date', y='Cum_GDD',source=source,line_width=2,line_color="blue")
    p.circle('Date','Cum_GDD',source=source)
    p.xaxis.axis_label_text_font_size = "20pt"
    p.yaxis.axis_label_text_font_size = "20pt"
    tab = Panel(child=p, title= k)
    return tab

## combine cumulative GDD into one plot

In [21]:
tab1 = plotTab(dfA,'BLOCK A');tab3 = plotTab(dfC,'BLOCK C');tab4 = plotTab(dfD,'BLOCK D')
layout = Tabs(tabs=[tab1, tab3, tab4]) 
show (layout)

<h1 style="font-family: Arial; font-size:2.6em;color:purple; font-style:bold; text-align: center;"><br>
Thank You </h1>
*******
   ###  github : denisDPR
   
   

   ### twitter ; denispastoty